<a href="https://colab.research.google.com/github/ArtemChemist/CFU_counter/blob/main/YOLO_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
from datetime import datetime
import time
import cv2
import shutil

from ultralytics import YOLO

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


# Import custom module with helper functions
import sys
sys.path.append('/content/drive/My Drive')
import funcs_yolo as f

Mounted at /content/drive


In [ ]:
# Unzip archives with images
!unzip -q "/content/drive/MyDrive/raw_train_val.zip" -d "/content"
!unzip -q "/content/drive/MyDrive/raw_train.zip" -d "/content/train"

In [ ]:
p = os.path.abspath('.')
input_dir = os.path.join(p, 'raw_test_val')
output_dir = os.path.join(p, 'drive', 'MyDrive', 'results')
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [ ]:
model_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'runs', 'detect', 'train', 'weights', 'best.pt' )
model = YOLO(model_path)

In [ ]:
expert_lbls = {}

ref_path = os.path.join(p, 'train', 'raw', 'images')
train_ims = []
for file in glob.glob(f'{ref_path}/*.jpg'):
    train_ims.append(file.split('/')[-1])

colors = {0: (0, 0, 0),
          1: (255, 0, 0),
          2: (0, 255, 0),
          3: (0, 0, 255),
          4: (50, 50, 50),
          5: (100, 0, 0),
          6: (0, 0, 100),
          7: (0, 100, 0),
          8: (70, 70, 70)}

cls_names = {0: 'artifact',
            1: 'bubble',
            2: 'colony',
            3: 'gate',
            4: 'lock',
            5: 'sharpie',
            6: 'star',
            7: 'tape',
            8: 'unlock'}

for file_path in glob.glob(f'{input_dir}/*.jpg'):
    filename = file_path.split('/')[-1]
    num_colonies = int(filename.split('-')[2].strip().split('.')[0])
    if filename not in train_ims:
      result = model(file_path, verbose = False)  # return a list of Results objects
      if (num_colonies > 0) != (2.0 in result[0].boxes.cls):
        print(file_path, num_colonies)
        expert_lbls[filename] = num_colonies
        pic = cv2.imread(file_path)
        for box, cls in zip(result[0].boxes.xyxy, result[0].boxes.cls):
            xB = int(box[2])
            xA = int(box[0])
            yB = int(box[3])
            yA = int(box[1])
            cv2.rectangle(pic, (xA, yA), (xB, yB), colors[int(cls)], 3)
        cv2.imwrite(os.path.join(output_dir, filename), pic)

/content/raw_test_val/6293301f3e8f37616ea3eae3aae6a076-170-0.jpg 0
/content/raw_test_val/d7104471de3ce0383668678a7c9abf49-242-0.jpg 0
/content/raw_test_val/acf465d76a98056a9a5e5a78b64bc4a9-99-0.jpg 0
/content/raw_test_val/ebcc918f42365bc298dd8ed1a63f2d84-99-0.jpg 0
/content/raw_test_val/8fdc3cf60488517646a09706290c26b1-270-0.jpg 0
/content/raw_test_val/97264f8cdb07fde4d54f812a993c64f8-198-0.jpg 0
/content/raw_test_val/86b94b31d97f19e56174de40950c7743-121-0.jpg 0
/content/raw_test_val/6f0bdfa78ac1c960e9a16c6fafcad29b-318-0.jpg 0
/content/raw_test_val/b69e1cf6960e56e9e155438496ae4771-101-0.jpg 0
/content/raw_test_val/c83afc2c009d788d822a1f30abc23511-339-0.jpg 0
/content/raw_test_val/2f6d325b6a883b971630691b4e613b9a-102-0.jpg 0
/content/raw_test_val/973422aaf00ce063942c0c86a95294ce-117-0.jpg 0
/content/raw_test_val/7b3c85a07ade57b7117ee005678769fa-199-0.jpg 0
/content/raw_test_val/5cc0bb47201ed932a95b5a807368cfdb-69-0.jpg 0
/content/raw_test_val/dbe31b6eb6b83c48f6f68b3b7cd1201d-190-0.jpg 

KeyboardInterrupt: 

In [ ]:
output_dir

'/content/drive/MyDrive/results'

In [ ]:
result[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([8., 4.], device='cuda:0')
conf: tensor([0.8843, 0.8146], device='cuda:0')
data: tensor([[1.0824e+03, 3.0049e+02, 1.2960e+03, 5.5008e+02, 8.8433e-01, 8.0000e+00],
        [6.5027e+02, 1.1622e+03, 9.4178e+02, 1.2887e+03, 8.1457e-01, 4.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1459, 1506)
shape: torch.Size([2, 6])
xywh: tensor([[1189.2036,  425.2823,  213.5372,  249.5859],
        [ 796.0256, 1225.4346,  291.5060,  126.4482]], device='cuda:0')
xywhn: tensor([[0.7896, 0.2915, 0.1418, 0.1711],
        [0.5286, 0.8399, 0.1936, 0.0867]], device='cuda:0')
xyxy: tensor([[1082.4349,  300.4894, 1295.9722,  550.0753],
        [ 650.2726, 1162.2104,  941.7786, 1288.6587]], device='cuda:0')
xyxyn: tensor([[0.7187, 0.2060, 0.8605, 0.3770],
        [0.4318, 0.7966, 0.6254, 0.8832]], device='cuda:0')

In [ ]:
result[0].names

{0: 'artifact',
 1: 'bubble',
 2: 'colony',
 3: 'gate',
 4: 'lock',
 5: 'sharpie',
 6: 'star',
 7: 'tape',
 8: 'unlock'}